In [1]:
V = 1 + (ord('С') + ord('А')) % 10
V

8

In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.inspection import permutation_importance
from collections import Counter, defaultdict

In [3]:
df = pd.read_csv('tweet_emotions.csv')
df.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [4]:
df.shape

(40000, 3)

In [5]:
df['sentiment'].nunique()

13

In [6]:
#подготавливаем данные для класссификационных моделей
df_clean = df.drop('tweet_id', axis=1)

#смотрим пропущенные значения
print(df_clean.isnull().sum())

sentiment    0
content      0
dtype: int64


In [7]:
#очистка текста
df_clean['content_clean'] = df_clean['content'].apply(
    lambda text: ' '.join(
        re.sub(r'(@\w+|http\S+)', '', text).split()
    )
)
#готовим обучающую и тестовую выборки
X = df_clean['content_clean']
y = df_clean['sentiment']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [8]:
#определяем способы предобработки
def create_vectorizers():
    #все слова с учетом регистра
    vectorizer1 = CountVectorizer(lowercase=False, binary=True)
    
    #слова в нижнем регистре
    vectorizer2 = CountVectorizer(lowercase=True, binary=True)
    
    #удаление стоп-слов
    vectorizer3 = CountVectorizer(lowercase=True, stop_words=list(english_stopwords), binary=True)
    
    #специфические слова
    vectorizer4 = CountVectorizer(lowercase=True, binary=True)
    
    return {
        'С учетом регистра': vectorizer1,
        'Нижний регистр': vectorizer2,
        'Без стоп-слов': vectorizer3,
        'Специфичные слова': vectorizer4 
    }

In [9]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
english_stopwords = set(stopwords.words('english'))

vectorizers = create_vectorizers()

print("Размерность признаков для разных способов предобработки:")
for name, vectorizer in vectorizers.items():
    if name != 'Специфичные слова':
        X_vec = vectorizer.fit_transform(X_train)
        print(f"{name}: {X_vec.shape[1]} признаков")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Анастасия\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Размерность признаков для разных способов предобработки:
С учетом регистра: 33550 признаков
Нижний регистр: 26537 признаков
Без стоп-слов: 26395 признаков


In [10]:
#ищем специфические слова
def find_class_specific_tokens(texts, labels, threshold=0.8, lowercase=True):
    class_token_counts = defaultdict(Counter)
    total_token_counts = Counter()
    
    for t, y in zip(texts, labels):
        toks = t.split()
        if lowercase:
            toks = [tok.lower() for tok in toks]  
        class_token_counts[y].update(toks)
        total_token_counts.update(toks)
    
    classes = list(class_token_counts.keys())
    specific = []
    
    for tok, total in total_token_counts.items():
        if total == 0:
            continue
        max_share = max(class_token_counts[c][tok] for c in classes) / total
        if max_share >= threshold:
            specific.append(tok)
    
    return sorted(set(specific))

#находим специфичные слова на обучающей выборке
specific_tokens = find_class_specific_tokens(X_train, y_train, threshold=0.8)

vectorizers['Специфичные слова'] = CountVectorizer(
    lowercase=True,      
    binary=True,
    vocabulary=specific_tokens
)

X_vec = vectorizers['Специфичные слова'].fit_transform(X_train) 
print(f"Специфичные слова: {X_vec.shape[1]} признаков")


Специфичные слова: 34052 признаков


In [11]:
vectorizers = {
    'С учетом регистра': CountVectorizer(lowercase=False, binary=True),
    'Нижний регистр': CountVectorizer(lowercase=True, binary=True),
    'Без стоп-слов': CountVectorizer(lowercase=True, stop_words=list(english_stopwords), binary=True),
    'Специфичные слова': CountVectorizer(lowercase=True, binary=True, vocabulary=specific_tokens),
}

models = {
    'RandomForest': RandomForestClassifier(n_estimators=50, max_depth=20, random_state=42, n_jobs=-1),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'SVM': LinearSVC(random_state=42, max_iter=1000),  
    'Naive Bayes': MultinomialNB()
}

results = {}

#обучение и оценка для каждой комбинации векторзатора и модели
for vec_name, vectorizer in vectorizers.items():
    print(f"Векторизатор: {vec_name}")
    
    #векторизация
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)
    
    print(f"Количество признаков: {X_train_vec.shape[1]}")
    
    results[vec_name] = {}
    
    for model_name, model in models.items():
        print(f"\nМодель: {model_name}")
        
        model.fit(X_train_vec, y_train)
        
        y_pred = model.predict(X_test_vec)
        
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        
        print(f"Accuracy: {accuracy:.4f}")
        print(f"F1-score: {f1:.4f}")
        
        results[vec_name][model_name] = {
            'accuracy': accuracy,
            'f1_score': f1,
            'num_features': X_train_vec.shape[1]
        }

Векторизатор: С учетом регистра
Количество признаков: 33550

Модель: RandomForest
Accuracy: 0.2795
F1-score: 0.1797

Модель: Logistic Regression
Accuracy: 0.3372
F1-score: 0.3108

Модель: SVM
Accuracy: 0.2911
F1-score: 0.2795

Модель: Naive Bayes
Accuracy: 0.3185
F1-score: 0.2629
Векторизатор: Нижний регистр
Количество признаков: 26537

Модель: RandomForest
Accuracy: 0.2812
F1-score: 0.1840

Модель: Logistic Regression
Accuracy: 0.3449
F1-score: 0.3183

Модель: SVM
Accuracy: 0.2985
F1-score: 0.2858

Модель: Naive Bayes
Accuracy: 0.3234
F1-score: 0.2712
Векторизатор: Без стоп-слов
Количество признаков: 26395

Модель: RandomForest
Accuracy: 0.2756
F1-score: 0.1799

Модель: Logistic Regression
Accuracy: 0.3392
F1-score: 0.3122

Модель: SVM
Accuracy: 0.2966
F1-score: 0.2828

Модель: Naive Bayes
Accuracy: 0.3225
F1-score: 0.2743
Векторизатор: Специфичные слова
Количество признаков: 34052

Модель: RandomForest
Accuracy: 0.2189
F1-score: 0.0850

Модель: Logistic Regression
Accuracy: 0.2245
F1

In [12]:
def print_results_tables(results):
    data = []
    for vec_name, models_dict in results.items():
        for model_name, metrics in models_dict.items():
            data.append({
                'Метод предобработки': vec_name,
                'Модель': model_name,
                'Accuracy': f"{metrics['accuracy']:.4f}",
                'F1-Score': f"{metrics['f1_score']:.4f}",
                'Кол-во признаков': metrics['num_features']
            })
    
    df_results = pd.DataFrame(data)
    
    print(f"\nСводная таблица результатов")
    print(df_results.to_string(index=False))
    
    print(f"\nТаблица accuracy по моделям и методам предобработки")
    pivot_acc = df_results.pivot(index='Метод предобработки', columns='Модель', values='Accuracy')
    print(pivot_acc.to_string())

    print(f"\nТаблица F1-score по моделям и методам предобработки")
    pivot_f1 = df_results.pivot(index='Метод предобработки', columns='Модель', values='F1-Score')
    print(pivot_f1.to_string())
    
    return df_results

df_results = print_results_tables(results)


Сводная таблица результатов
Метод предобработки              Модель Accuracy F1-Score  Кол-во признаков
  С учетом регистра        RandomForest   0.2795   0.1797             33550
  С учетом регистра Logistic Regression   0.3372   0.3108             33550
  С учетом регистра                 SVM   0.2911   0.2795             33550
  С учетом регистра         Naive Bayes   0.3185   0.2629             33550
     Нижний регистр        RandomForest   0.2812   0.1840             26537
     Нижний регистр Logistic Regression   0.3449   0.3183             26537
     Нижний регистр                 SVM   0.2985   0.2858             26537
     Нижний регистр         Naive Bayes   0.3234   0.2712             26537
      Без стоп-слов        RandomForest   0.2756   0.1799             26395
      Без стоп-слов Logistic Regression   0.3392   0.3122             26395
      Без стоп-слов                 SVM   0.2966   0.2828             26395
      Без стоп-слов         Naive Bayes   0.3225   0.2743  

In [13]:
def analyze_all_models_feature_importance(results, vectorizers, X_train, y_train, X_test, y_test, top_n=15):
    print("Анализ важных признаков для всех алгоритмов")
    
    feature_analysis = {}
    
    for vec_name, vectorizer in vectorizers.items():
        print(f"\nМетод предобработки: {vec_name}")
        
        X_train_vec = vectorizer.fit_transform(X_train)
        X_test_vec = vectorizer.transform(X_test)
        feature_names = vectorizer.get_feature_names_out()
        
        feature_analysis[vec_name] = {}
        
        for model_name, model in models.items():
            print(f"\n--- Модель: {model_name} ---")
            print(f"Количество признаков: {len(feature_names)}")
            
            model.fit(X_train_vec, y_train)
            
            importance_info = get_detailed_feature_importance(
                model, feature_names, X_test_vec, y_test, top_n=top_n
            )
            
            feature_analysis[vec_name][model_name] = importance_info
            
            #выводим топ важных признаков
            if isinstance(importance_info, dict): 
                for class_label, features in importance_info.items():
                    print(f"  Класс '{class_label}' - Топ-{top_n} важных признаков:")
                    for i, (feature, score) in enumerate(features, 1):
                        print(f"    {i:2d}. {feature:20s} : {score:+.4f}")
            else:  
                print(f"  Топ-{top_n} важных признаков:")
                for i, (feature, score) in enumerate(importance_info, 1):
                    print(f"    {i:2d}. {feature:20s} : {score:.4f}")
            
            print() 
    
    return feature_analysis

def get_detailed_feature_importance(model, feature_names, X_test, y_test, top_n=15):

    if hasattr(model, 'feature_importances_'):  # Random Forest
        importance = model.feature_importances_
        indices = np.argsort(importance)[::-1][:top_n]
        return [(feature_names[i], importance[i]) for i in indices]
    
    elif hasattr(model, 'coef_'):  #линейные модели 
        coef = model.coef_
        if len(coef.shape) == 1: 
            importance = np.abs(coef)
            indices = np.argsort(importance)[::-1][:top_n]
            return [(feature_names[i], coef[i]) for i in indices]
        else: 
            results = {}
            for i, class_label in enumerate(model.classes_):
                importance = np.abs(coef[i])
                indices = np.argsort(importance)[::-1][:top_n]
                results[class_label] = [(feature_names[j], coef[i][j]) for j in indices]
            return results
    
    elif hasattr(model, 'feature_log_prob_'): 
        #для Naive Bayes используем разность логарифмических вероятностей
        if len(model.classes_) == 2: 
            importance = model.feature_log_prob_[1] - model.feature_log_prob_[0]
            indices = np.argsort(np.abs(importance))[::-1][:top_n]
            return [(feature_names[i], importance[i]) for i in indices]
        else: 
            results = {}
            for i, class_label in enumerate(model.classes_):
                importance = model.feature_log_prob_[i]
                indices = np.argsort(importance)[::-1][:top_n]
                results[class_label] = [(feature_names[j], importance[j]) for j in indices]
            return results
    
    else:
        from sklearn.inspection import permutation_importance
        perm_importance = permutation_importance(
            model, X_test, y_test, n_repeats=5, random_state=42
        )
        importance = perm_importance.importances_mean
        indices = np.argsort(importance)[::-1][:top_n]
        return [(feature_names[i], importance[i]) for i in indices]

feature_analysis_results = analyze_all_models_feature_importance(
    results, vectorizers, X_train, y_train, X_test, y_test, top_n=5
)

Анализ важных признаков для всех алгоритмов

Метод предобработки: С учетом регистра

--- Модель: RandomForest ---
Количество признаков: 33550
  Топ-5 важных признаков:
     1. sad                  : 0.0219
     2. love                 : 0.0167
     3. Happy                : 0.0162
     4. Day                  : 0.0120
     5. day                  : 0.0093


--- Модель: Logistic Regression ---
Количество признаков: 33550
  Класс 'anger' - Топ-5 важных признаков:
     1. soon                 : +1.3146
     2. everything           : +1.3043
     3. knows                : +1.2382
     4. library              : +1.1642
     5. annoying             : +1.1377
  Класс 'boredom' - Топ-5 важных признаков:
     1. bored                : +3.0202
     2. boring               : +1.9367
     3. minutes              : +1.9196
     4. stuck                : +1.7374
     5. BORED                : +1.5376
  Класс 'empty' - Топ-5 важных признаков:
     1. Bored                : +1.8173
     2. bored      

In [14]:
#поиск уникальных слов
def find_class_unique_tokens(texts, labels, lowercase=True, token_pattern=r"(?u)\b\w+\b", min_df_docs=1):

    vec = CountVectorizer(lowercase=lowercase, token_pattern=token_pattern, binary=True)
    X_bin = vec.fit_transform(texts)
    feature_names = np.array(vec.get_feature_names_out())
    labels_arr = np.array(labels)

    #подсчет общей частоты слов
    df_total = np.asarray(X_bin.sum(axis=0)).ravel()

    unique_tokens_by_class = {}
    for cls in np.unique(labels_arr):
        idx_cls = np.where(labels_arr == cls)[0]
        X_cls = X_bin[idx_cls]
        df_cls = np.asarray(X_cls.sum(axis=0)).ravel()
        #слово встречается только в этом классе и нигде больше
        mask_unique = (df_cls >= min_df_docs) & (df_total == df_cls)
        toks = feature_names[mask_unique]
        unique_tokens_by_class[cls] = sorted(toks.tolist())
    return unique_tokens_by_class

unique_tokens_by_class = find_class_unique_tokens(
    X_train.tolist(), y_train.tolist(),
    lowercase=True, token_pattern=r"(?u)\b\w+\b", min_df_docs=1
)

unique_vocab_all = sorted({tok for toks in unique_tokens_by_class.values() for tok in toks})
print(f"Уникальные слова, всего признаков: {len(unique_vocab_all)}")
for cls, toks in unique_tokens_by_class.items():
    print(f"Класс '{cls}': {len(toks)} уникальных слов")

vectorizers['Уникальные слова'] = CountVectorizer(
    lowercase=True,
    binary=True,
    vocabulary=unique_vocab_all
)

#прогоняем все модели на уникальных словах
vec_name = 'Уникальные слова'
vectorizer = vectorizers[vec_name]

X_train_unique = vectorizer.fit_transform(X_train) 
X_test_unique  = vectorizer.transform(X_test)

results[vec_name] = {}
for model_name, model in models.items():
    print(f"\n--- Модель: {model_name} ---")
    model.fit(X_train_unique, y_train)
    y_pred = model.predict(X_test_unique)
    acc = accuracy_score(y_test, y_pred)
    f1  = f1_score(y_test, y_pred, average='weighted')
    print(f"Accuracy: {acc:.4f} | F1: {f1:.4f} | Признаков: {X_train_unique.shape[1]}")

    #важность признаков для анализа участия уникальных признаков
    fi = get_detailed_feature_importance(
        model,
        vectorizer.get_feature_names_out(),
        X_test_unique,
        y_test,
        top_n=15
    )

    results[vec_name][model_name] = {
        'accuracy': acc,
        'f1_score': f1,
        'num_features': X_train_unique.shape[1],
        'feature_importance': fi
    }

df_results = print_results_tables(results)

Уникальные слова, всего признаков: 16181
Класс 'anger': 58 уникальных слов
Класс 'boredom': 66 уникальных слов
Класс 'empty': 338 уникальных слов
Класс 'enthusiasm': 322 уникальных слов
Класс 'fun': 870 уникальных слов
Класс 'happiness': 2119 уникальных слов
Класс 'hate': 698 уникальных слов
Класс 'love': 1387 уникальных слов
Класс 'neutral': 3572 уникальных слов
Класс 'relief': 537 уникальных слов
Класс 'sadness': 1929 уникальных слов
Класс 'surprise': 906 уникальных слов
Класс 'worry': 3379 уникальных слов

--- Модель: RandomForest ---
Accuracy: 0.2166 | F1: 0.0792 | Признаков: 16181

--- Модель: Logistic Regression ---
Accuracy: 0.2167 | F1: 0.1163 | Признаков: 16181

--- Модель: SVM ---
Accuracy: 0.2109 | F1: 0.1224 | Признаков: 16181

--- Модель: Naive Bayes ---
Accuracy: 0.2161 | F1: 0.1159 | Признаков: 16181

Сводная таблица результатов
Метод предобработки              Модель Accuracy F1-Score  Кол-во признаков
  С учетом регистра        RandomForest   0.2795   0.1797           